In [2]:
from torch.utils.cpp_extension import load_inline
import torch

cuda_begin = r'''
#include <torch/extension.h>
#include <stdio.h>
#include <c10/cuda/CUDAException.h>

#define CHECK_CUDA(x) TORCH_CHECK(x.device().is_cuda(), #x " must be a CUDA tensor")
#define CHECK_CONTIGUOUS(x) TORCH_CHECK(x.is_contiguous(), #x " must be contiguous")
#define CHECK_INPUT(x) CHECK_CUDA(x); CHECK_CONTIGUOUS(x)
#define CUDA_ERR(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true)
{
   if (code != cudaSuccess) 
   {
      fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) exit(code);
   }
}
__host__ __device__ inline unsigned int cdiv(unsigned int a, unsigned int b) { return (a+b-1)/b;}
'''

def load_cuda(cuda_src, cpp_src, funcs, opt=True, verbose=False, name=None):
    "Simple wrapper for torch.utils.cpp_extension.load_inline"
    if name is None: name = funcs[0]
    flags = "-O3 -Xptxas -O3 -Xcompiler -O3" if opt else "-O0 -Xptxas -O0 -Xcompiler -O0"
    return load_inline(cuda_sources=[cuda_src], cpp_sources=[cpp_src], functions=funcs,
                       extra_cuda_cflags=[flags], verbose=verbose, name=name)

def cdiv(a,b):
    "Int ceiling division of `a` over `b`"
    return (a+b-1)//b

torch.manual_seed(42)

In [54]:
cuda_src = cuda_begin + r'''
__global__ void reduce_kernel(float* m, float* output) {
    unsigned int i = threadIdx.x;
    for(unsigned int stride=blockDim.x; stride >= 1; stride /= 2) {
        if (threadIdx.x < stride) {
            m[i] = max( m[i+stride], m[i]);
        }
        __syncthreads();
    }
    if(threadIdx.x == 0) *output = m[0];
}

torch::Tensor reduce(torch::Tensor m) {
    CHECK_INPUT(m);
    auto output = torch::zeros({1}, m.options());

    reduce_kernel<<<1, 32>>>( m.data_ptr<float>(), output.data_ptr<float>());

    C10_CUDA_KERNEL_LAUNCH_CHECK();
    return output;
}
'''

cpp_src = "torch::Tensor reduce(torch::Tensor m);"

In [55]:
module = load_cuda(cuda_src, cpp_src, ["reduce"])

In [56]:
x = torch.randn(32).contiguous().cuda()

In [63]:
x.max()

tensor(1.9892, device='cuda:0')

In [72]:
x

tensor([1.9892, 1.9892, 0.8539, 1.9892, 0.5397, 0.5655, 0.5058, 1.9892, 1.2554,
        0.0000, 0.8539, 0.5130, 0.5397, 0.5655, 0.5058, 0.4097, 0.0000, 0.2762,
        0.0000, 0.4626, 0.0000, 0.0000, 0.0000, 1.4639, 1.2554, 0.0000, 0.8539,
        0.5130, 0.5397, 0.5655, 0.5058, 0.2225], device='cuda:0')

In [68]:
module.reduce(x)

tensor([1.9892], device='cuda:0')

In [60]:
x.sum()

tensor(21.7233, device='cuda:0')